In [1]:
from keras_tuner import HyperModel
import keras_tuner as kt

import numpy as np
import pandas as pd
#!pip install keras_tuner
#import keras_tuner

import keras
from keras import models, layers, metrics, Sequential
from keras.layers import Dense, BatchNormalization, Dropout
import tensorflow as tf
from keras.regularizers import l1,l2,l1_l2
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Input, Dense, Multiply, Concatenate
from tensorflow.keras.models import Model

import tensorflow as tf
import random

import keras_tuner
from keras_tuner import HyperModel
import keras_tuner as kt

from sklearn.preprocessing import MinMaxScaler

2023-09-29 11:04:51.570946: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


c

## 49 Variables [only firm-level characteristics]

In [2]:
char_core = [
          'acc','agr','beta','bm','cash','cashpr','cfp','chatoia','chcsho','chfeps','chinv',
          'chmom','chpmia','chtx','currat','depr','dy','ear','ep','gma','grcapx','grltnoa',
          'ill','indmom','invest','lev','lgr','maxret','mom12m','mom1m','mom36m','mve','nincr',
          'orgcap','pchgm_pchsale','pchsale_pchinvt','pchsale_pchrect','pchsale_pchxsga',
          'retvol','roaq','roavol','roeq','salecash','saleinv','sgr','sp','std_dolvol','std_turn','turn']

#merged_latent.to_pickle('merged_latent49.pkl')
merged_latent49 = pd.read_pickle("merged_latent49.pkl")

# merged data 1
firm_col = merged_latent49.columns.tolist()[18:18+49]
latent_col = merged_latent49.columns.tolist()[18+49+1:]
print(len(firm_col), len(latent_col))

49 49


In [3]:
df_train = merged_latent49[merged_latent49.index < pd.Period((str(1995)+"-1"),freq='M')]
df_val = merged_latent49[(merged_latent49.index >= pd.Period((str(1995)+"-1"),freq='M')) & (merged_latent49.index < pd.Period((str(2000)+"-1"),freq='M'))]
df_test = merged_latent49[merged_latent49.index >= pd.Period((str(2000)+"-1"),freq='M')]

merged_train_firm = df_train[firm_col]
merged_train_latent = df_train[latent_col]
merged_train_y = df_train['predicted_return']
print(merged_train_firm.shape, merged_train_latent.shape, merged_train_y.shape)

merged_val_firm = df_val[firm_col]
merged_val_latent = df_val[latent_col]
merged_val_y = df_val['predicted_return']
print(merged_val_firm.shape, merged_val_latent.shape, merged_val_y.shape)

merged_test_firm = df_test[firm_col]
merged_test_latent = df_test[latent_col]
merged_test_y = df_test['predicted_return']
print(merged_test_firm.shape, merged_test_latent.shape, merged_test_y.shape)

(88347, 49) (88347, 49) (88347,)
(31851, 49) (31851, 49) (31851,)
(90007, 49) (90007, 49) (90007,)


---

# Model Design

1. Multi-input with conditional autoencoder
2. Multi-input with standard autoencoder

In [4]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den

def multi_input_model(num_input):
    dropout =0.
    l1_reg = 0.0
    l2_reg =0.0
    input1 = Input(shape=(num_input,))
    input2 = Input(shape=(num_input,))
    x1 = Dense(4, activation='relu')(input2)
    x1 = Dense(num_input, activation='relu')(x1)

    merged = Multiply()([input1, x1])
    x2 = Dense(32, activation='relu')(merged)
    x2 = Dense(16, activation='relu')(x2)
    #x2 = Dense(8, activation='relu')(x2)
    output = Dense(1, activation='linear')(x2)

    model = Model(inputs=[input1, input2], outputs=output)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.92) 
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[my_metric_fn])
    return model 

def Autoencoder(num_input,lr=0.0001):
    input_img = Input(shape=(num_input, ))
    encoded1 = Dense(4, activation='sigmoid')(input_img)
    decoded1 = Dense(num_input, activation='sigmoid')(encoded1)
    
    autoencoder = Model(inputs=input_img, outputs=decoded1)
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    autoencoder.compile(loss='mean_squared_error', optimizer=opt, metrics=[my_metric_fn])
    return autoencoder

In [5]:
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()
# 10 Trials for Prediction
merged_train_latent_all = np.concatenate((merged_train_latent, merged_val_latent))
merged_train_firm_all = np.concatenate((merged_train_firm, merged_val_firm))
merged_y_all = np.concatenate((merged_train_y, merged_val_y))

for i,random_seed in enumerate(seed_list):
    print(i)
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    model = multi_input_model(num_input=merged_train_firm.shape[1])
    model.fit([merged_train_firm, merged_train_latent], merged_train_y,
              validation_data=([merged_val_firm, merged_val_latent], merged_val_y),
              epochs=30,batch_size = 128,
              callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)])
    print(model.evaluate([merged_test_firm, merged_test_latent], merged_test_y))
    y_hat = model.predict([merged_test_firm, merged_test_latent]).reshape(-1)
    yhat_df[random_seed] = y_hat
    
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0


2023-09-29 11:05:53.380630: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/30
691/691 [==============================] - 3s 4ms/step - loss: 0.0121 - my_metric_fn: 0.0056 - val_loss: 0.0146 - val_my_metric_fn: 0.0058
Epoch 2/30
691/691 [==============================] - 2s 3ms/step - loss: 0.0120 - my_metric_fn: 0.0117 - val_loss: 0.0146 - val_my_metric_fn: 0.0061
Epoch 3/30
691/691 [==============================] - 2s 3ms/step - loss: 0.0119 - my_metric_fn: 0.0212 - val_loss: 0.0145 - val_my_metric_fn: 0.0095
Epoch 4/30
691/691 [==============================] - 2s 3ms/step - loss: 0.0116 - my_metric_fn: 0.0446 - val_loss: 0.0141 - val_my_metric_fn: 0.0282
Epoch 5/30
691/691 [==============================] - 2s 3ms/step - loss: 0.0112 - my_metric_fn: 0.0828 - val_loss: 0.0139 - val_my_metric_fn: 0.0378
Epoch 6/30
691/691 [==============================] - 2s 3ms/step - loss: 0.0108 - my_metric_fn: 0.1075 - val_loss: 0.0139 - val_my_metric_fn: 0.0282
Epoch 7/30
2813/2813 [==============================] - 5s 2ms/step - loss: 0.0141 - my_metric_fn: 0

In [6]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")
yhat_df.to_pickle("model1a49.pkl")

R^2 is 0.15328355096760926


----

In [9]:
import random 
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)
    
    autoencoder = Autoencoder(num_input=merged_train_firm.shape[1])
    autoencoder.fit(merged_train_latent, merged_train_latent,
                epochs = 20, batch_size = 216,
                validation_data = (merged_val_latent, merged_val_latent),
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)])
 
    model = multi_input_model(num_input=merged_train_firm.shape[1])
    model.layers[1].set_weights(autoencoder.layers[1].get_weights())
    #model.layers[3].set_weights(autoencoder.layers[2].get_weights())
    model.layers[1].trainable = False
   # model.layers[3].trainable = False
    model.fit([merged_train_latent, merged_train_firm], merged_train_y,
              epochs=50,
              validation_data=([merged_val_latent, merged_val_firm], merged_val_y),
              callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)])
    print(model.evaluate([merged_test_latent, merged_test_firm], merged_test_y))
    y_hat = model.predict([merged_test_latent, merged_test_firm]).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

Epoch 1/20
410/410 [==============================] - 2s 4ms/step - loss: 0.2424 - my_metric_fn: -380.2728 - val_loss: 0.2277 - val_my_metric_fn: -488.7396
Epoch 2/20
410/410 [==============================] - 1s 2ms/step - loss: 0.2138 - my_metric_fn: -333.8434 - val_loss: 0.2001 - val_my_metric_fn: -429.1965
Epoch 3/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1869 - my_metric_fn: -292.0875 - val_loss: 0.1742 - val_my_metric_fn: -373.2765
Epoch 4/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1618 - my_metric_fn: -253.3098 - val_loss: 0.1504 - val_my_metric_fn: -321.9242
Epoch 5/20
410/410 [==============================] - 1s 2ms/step - loss: 0.1390 - my_metric_fn: -216.5074 - val_loss: 0.1290 - val_my_metric_fn: -275.7303
Epoch 6/20
410/410 [==============================] - 1s 2ms/step - loss: 0.1186 - my_metric_fn: -184.7386 - val_loss: 0.1101 - val_my_metric_fn: -234.8516
Epoch 7/20
410/410 [==============================] - 1s 2ms/ste

410/410 [==============================] - 1s 2ms/step - loss: 0.0663 - my_metric_fn: -102.9869 - val_loss: 0.0618 - val_my_metric_fn: -130.9259
Epoch 10/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0557 - my_metric_fn: -86.2642 - val_loss: 0.0522 - val_my_metric_fn: -110.0087
Epoch 11/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0467 - my_metric_fn: -72.1141 - val_loss: 0.0440 - val_my_metric_fn: -92.3151
Epoch 12/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0392 - my_metric_fn: -60.2939 - val_loss: 0.0370 - val_my_metric_fn: -77.3997
Epoch 13/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0329 - my_metric_fn: -50.4823 - val_loss: 0.0312 - val_my_metric_fn: -64.8564
Epoch 14/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0275 - my_metric_fn: -42.0555 - val_loss: 0.0263 - val_my_metric_fn: -54.3246
Epoch 15/20
410/410 [==============================] - 1s 2ms/step - loss: 0.02

410/410 [==============================] - 1s 2ms/step - loss: 0.0388 - my_metric_fn: -59.7632 - val_loss: 0.0371 - val_my_metric_fn: -77.1125
Epoch 13/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0326 - my_metric_fn: -50.0344 - val_loss: 0.0312 - val_my_metric_fn: -64.6253
Epoch 14/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0273 - my_metric_fn: -41.6709 - val_loss: 0.0264 - val_my_metric_fn: -54.1441
Epoch 15/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0229 - my_metric_fn: -34.7673 - val_loss: 0.0223 - val_my_metric_fn: -45.3540
Epoch 16/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0192 - my_metric_fn: -29.0916 - val_loss: 0.0188 - val_my_metric_fn: -37.9878
Epoch 17/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0161 - my_metric_fn: -24.2111 - val_loss: 0.0159 - val_my_metric_fn: -31.8173
Epoch 18/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0136 

In [11]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")
yhat_df.to_pickle("model1b49.pkl")

R^2 is 0.1394947655677452


---

## 103 Variables [only firm-level characteristics]

char_core = ['absacc','acc','aeavol','age','agr','beta','betasq','bm','bm_ia',
                'cash','cashdebt','cashpr','cfp','cfp_ia','chatoia','chcsho','chempia',
                            'chinv','chmom','chpmia','chtx','cinvest','convind','currat','depr',
                            'divi','divo','dolvol','dy','ear','egr','ep','gma','grcapx','grltnoa',
                            'herf','hire','idiovol','indmom','invest','ipo','lev','lgr','mom12m',
                            'mom1m','mom36m','mom6m','ms','mve','mve_ia','nincr','operprof','orgcap',
                            'pchcapx_ia','pchcurrat','pchdepr','pchgm_pchsale','pchquick','pchsale_pchinvt',
                            'pchsale_pchrect','pchsale_pchxsga','pchsaleinv','pctacc','pricedelay','ps',
                            'quick','rd','rd_mve','rd_sale','roaq','roavol','roeq','roic','rsup','salecash',
                            'saleinv','salerec','securedind','sgr','sgrvol','sin','sp','stdacc','stdcf','sue',
                            'tang','tb','turn']


def import_firm(q=0.2, scaled = False):
    df_firm = pd.read_pickle('firm_df103.pkl')
    df_firm.index = pd.to_datetime(df_firm.index).to_period('M')
    # chars_data 
    chars_data = df_firm.copy()
    factors = chars_data.groupby('jdate').apply(
        lambda x: x[char_core].apply(lambda z: x[z >= z.quantile(1 - q)]['predicted_return'].mean() -
                    x[z <= z.quantile(q)]['predicted_return'].mean()))
    if scaled == True:
        scaler = MinMaxScaler(feature_range=(-0.5, 0.5))
        factors_scaled = scaler.fit_transform(factors)
        factors_scaled_df = pd.DataFrame(factors_scaled, columns=factors.columns)
        factors_scaled_df.index = factors.index
        merged_latent = pd.merge(df_firm, factors_scaled_df, left_on='jdate', right_on=factors_scaled_df.index, how='inner')
    elif scaled == False:
        merged_latent = pd.merge(df_firm, factors, left_on='jdate', right_on=factors.index, how='inner')
    merged_latent.index = df_firm.index
    return merged_latent

merged_latent = import_firm(q=0.2, scaled = False)
merged_latent.to_pickle("merged_latent103.pkl")
firm_col = merged_latent.columns.tolist()[16:16+88]
latent_col = merged_latent.columns.tolist()[16+88+1:]
print(len(firm_col), len(latent_col))

In [12]:
#merged_latent.to_pickle('merged_latent49.pkl')
merged_latent103 = pd.read_pickle("merged_latent103.pkl")
# merged data 1
firm_col = merged_latent103.columns.tolist()[16:16+88]
latent_col = merged_latent103.columns.tolist()[16+88+1:]
print(len(firm_col), len(latent_col))

88 88


In [15]:
df_train = merged_latent103[merged_latent103.index < pd.Period((str(1995)+"-1"),freq='M')]
df_val = merged_latent103[(merged_latent103.index >= pd.Period((str(1995)+"-1"),freq='M')) & (merged_latent103.index < pd.Period((str(2000)+"-1"),freq='M'))]
df_test = merged_latent103[merged_latent103.index >= pd.Period((str(2000)+"-1"),freq='M')]

merged_train_firm = df_train[firm_col]
merged_train_latent = df_train[latent_col]
merged_train_y = df_train['predicted_return']
print(merged_train_firm.shape, merged_train_latent.shape, merged_train_y.shape)

merged_val_firm = df_val[firm_col]
merged_val_latent = df_val[latent_col]
merged_val_y = df_val['predicted_return']
print(merged_val_firm.shape, merged_val_latent.shape, merged_val_y.shape)

merged_test_firm = df_test[firm_col]
merged_test_latent = df_test[latent_col]
merged_test_y = df_test['predicted_return']
print(merged_test_firm.shape, merged_test_latent.shape, merged_test_y.shape)

(946649, 88) (946649, 88) (946649,)
(412547, 88) (412547, 88) (412547,)
(1163746, 88) (1163746, 88) (1163746,)


In [16]:
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()
# 10 Trials for Prediction
merged_train_latent_all = np.concatenate((merged_train_latent, merged_val_latent))
merged_train_firm_all = np.concatenate((merged_train_firm, merged_val_firm))
merged_y_all = np.concatenate((merged_train_y, merged_val_y))

for i,random_seed in enumerate(seed_list):
    print(i)
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    model = multi_input_model(num_input=merged_train_firm.shape[1])
    model.fit([merged_train_firm, merged_train_latent], merged_train_y,
              validation_data=([merged_val_firm, merged_val_latent], merged_val_y),
              epochs=30,batch_size = 128,
              callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)])
    print(model.evaluate([merged_test_firm, merged_test_latent], merged_test_y))
    y_hat = model.predict([merged_test_firm, merged_test_latent]).reshape(-1)
    yhat_df[random_seed] = y_hat
    
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0
Epoch 1/30
7396/7396 [==============================] - 29s 4ms/step - loss: 0.0317 - my_metric_fn: 0.0428 - val_loss: 0.0411 - val_my_metric_fn: 0.0282
Epoch 2/30
7396/7396 [==============================] - 24s 3ms/step - loss: 0.0309 - my_metric_fn: 0.0668 - val_loss: 0.0402 - val_my_metric_fn: 0.0444
Epoch 3/30
7396/7396 [==============================] - 26s 4ms/step - loss: 0.0307 - my_metric_fn: 0.0748 - val_loss: 0.0400 - val_my_metric_fn: 0.0495
Epoch 4/30
7396/7396 [==============================] - 24s 3ms/step - loss: 0.0306 - my_metric_fn: 0.0789 - val_loss: 0.0398 - val_my_metric_fn: 0.0536
Epoch 5/30
7396/7396 [==============================] - 22s 3ms/step - loss: 0.0305 - my_metric_fn: 0.0828 - val_loss: 0.0397 - val_my_metric_fn: 0.0544
Epoch 6/30
7396/7396 [==============================] - 22s 3ms/step - loss: 0.0304 - my_metric_fn: 0.0853 - val_loss: 0.0396 - val_my_metric_fn: 0.0546
Epoch 7/30
7396/7396 [==============================] - 22s 3ms/step - loss: 0.0

Epoch 3/30
7396/7396 [==============================] - 21s 3ms/step - loss: 0.0308 - my_metric_fn: 0.0711 - val_loss: 0.0400 - val_my_metric_fn: 0.0494
Epoch 4/30
7396/7396 [==============================] - 22s 3ms/step - loss: 0.0307 - my_metric_fn: 0.0756 - val_loss: 0.0400 - val_my_metric_fn: 0.0511
Epoch 5/30
7396/7396 [==============================] - 22s 3ms/step - loss: 0.0306 - my_metric_fn: 0.0800 - val_loss: 0.0399 - val_my_metric_fn: 0.0537
Epoch 6/30
7396/7396 [==============================] - 21s 3ms/step - loss: 0.0305 - my_metric_fn: 0.0838 - val_loss: 0.0398 - val_my_metric_fn: 0.0551
Epoch 7/30
7396/7396 [==============================] - 21s 3ms/step - loss: 0.0304 - my_metric_fn: 0.0867 - val_loss: 0.0398 - val_my_metric_fn: 0.0561
Epoch 8/30
36368/36368 [==============================] - 58s 2ms/step - loss: 0.0351 - my_metric_fn: 0.0512
[0.035095080733299255, 0.051203832030296326]
36368/36368 [==============================] - 53s 1ms/step
9
Epoch 1/30
7396/739

In [17]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")
yhat_df.to_pickle("model1a103.pkl")

R^2 is 0.11015854419147508


In [18]:
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)
    
    autoencoder = Autoencoder(num_input=merged_train_firm.shape[1])
    autoencoder.fit(merged_train_latent, merged_train_latent,
                epochs = 20, batch_size = 216,
                validation_data = (merged_val_latent, merged_val_latent),
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)])
 
    model = multi_input_model(num_input=merged_train_firm.shape[1])
    model.layers[1].set_weights(autoencoder.layers[1].get_weights())
    #model.layers[3].set_weights(autoencoder.layers[2].get_weights())
    model.layers[1].trainable = False
   # model.layers[3].trainable = False
    model.fit([merged_train_latent, merged_train_firm], merged_train_y,
              epochs=50,
              validation_data=([merged_val_latent, merged_val_firm], merged_val_y),
              callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)])
    print(model.evaluate([merged_test_latent, merged_test_firm], merged_test_y))
    y_hat = model.predict([merged_test_latent, merged_test_firm]).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

Epoch 1/20
4383/4383 [==============================] - 14s 3ms/step - loss: 0.1273 - my_metric_fn: -264.1497 - val_loss: 0.0469 - val_my_metric_fn: -130.0382
Epoch 2/20
4383/4383 [==============================] - 12s 3ms/step - loss: 0.0199 - my_metric_fn: -40.4387 - val_loss: 0.0078 - val_my_metric_fn: -18.7040
Epoch 3/20
4383/4383 [==============================] - 12s 3ms/step - loss: 0.0033 - my_metric_fn: -5.8104 - val_loss: 0.0021 - val_my_metric_fn: -2.6939
Epoch 4/20
4383/4383 [==============================] - 12s 3ms/step - loss: 8.9041e-04 - my_metric_fn: -0.8361 - val_loss: 0.0013 - val_my_metric_fn: -0.3706
Epoch 5/20
4383/4383 [==============================] - 12s 3ms/step - loss: 5.3519e-04 - my_metric_fn: -0.0961 - val_loss: 0.0012 - val_my_metric_fn: -0.0345
Epoch 6/20
4383/4383 [==============================] - 13s 3ms/step - loss: 4.7728e-04 - my_metric_fn: 0.0240 - val_loss: 0.0011 - val_my_metric_fn: 0.0184
Epoch 7/20
4383/4383 [==============================] 

4383/4383 [==============================] - 12s 3ms/step - loss: 4.7667e-04 - my_metric_fn: 0.0253 - val_loss: 0.0011 - val_my_metric_fn: 0.0185
Epoch 7/20
4383/4383 [==============================] - 12s 3ms/step - loss: 4.5437e-04 - my_metric_fn: 0.0716 - val_loss: 0.0011 - val_my_metric_fn: 0.0518
Epoch 8/20
4383/4383 [==============================] - 12s 3ms/step - loss: 4.3901e-04 - my_metric_fn: 0.1033 - val_loss: 0.0011 - val_my_metric_fn: 0.0733
Epoch 9/20
4383/4383 [==============================] - 12s 3ms/step - loss: 4.2796e-04 - my_metric_fn: 0.1261 - val_loss: 0.0011 - val_my_metric_fn: 0.0857
Epoch 10/20
4383/4383 [==============================] - 12s 3ms/step - loss: 4.1931e-04 - my_metric_fn: 0.1441 - val_loss: 0.0011 - val_my_metric_fn: 0.0922
Epoch 11/20
4383/4383 [==============================] - 12s 3ms/step - loss: 4.1326e-04 - my_metric_fn: 0.1566 - val_loss: 0.0011 - val_my_metric_fn: 0.0966
Epoch 12/20
4383/4383 [==============================] - 12s 3ms/st

In [19]:
 # Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")
yhat_df.to_pickle("model1b103.pkl")

R^2 is 0.0986526735196317
